# Módulo `subprocess` - Ejecutar Comandos del Sistema

El módulo `subprocess` permite ejecutar comandos del sistema operativo y capturar su salida desde Python.

## 🎯 ¿Qué vamos a hacer en esta demo?

Aprenderemos a ejecutar comandos del sistema operativo desde Python, capturar su salida y manejar errores.

### 📋 Operaciones que realizaremos:

1. **Ejecutar comandos simples**
   - `subprocess.run()` - Ejecutar y esperar resultado
   - `capture_output=True` - Capturar stdout y stderr
   - `text=True` - Obtener strings en lugar de bytes

2. **Comandos Unix/Linux**
   - `ls -lh` - Listar archivos
   - Pipes: `ls -la | grep demo` - Encadenar comandos

3. **Uso del shell**
   - `shell=True` - Ejecutar con características del shell
   - ⚠️ **Riesgos de seguridad** (inyección de comandos)
   - Variables de entorno (`$USER`, `$HOME`)

4. **Control del proceso**
   - `subprocess.Popen()` - Control más fino
   - Salida en tiempo real (línea por línea)
   - Códigos de retorno (éxito/error)

5. **Timeouts y límites de tiempo**
   - `timeout=N` - Limitar tiempo de ejecución
   - `TimeoutExpired` - Manejo de comandos que tardan demasiado

6. **Información del sistema**
   - `hostname` - Nombre del equipo
   - `whoami` - Usuario actual

### 🔑 Parámetros clave de `subprocess.run()`:

| Parámetro | Descripción | Ejemplo |
|-----------|-------------|---------|
| `capture_output=True` | Captura stdout y stderr | `resultado.stdout` |
| `text=True` | Devuelve strings (no bytes) | Sin `decode()` |
| `check=True` | Excepción si falla | `CalledProcessError` |
| `timeout=N` | Límite de tiempo (segundos) | `timeout=5` |
| `shell=True` | ⚠️ Ejecuta en shell | Variables `$VAR` |

### 💡 ¿Cuándo usar `subprocess` vs `os`?

- **`os`**: Operaciones nativas de Python (crear carpetas, listar archivos)
- **`subprocess`**: Ejecutar comandos externos que no tienen equivalente en Python

---

## 1. Ejecutar Comando Simple

In [1]:
import subprocess

# Ejecutar comando simple
resultado = subprocess.run(['echo', 'Hola desde subprocess'], 
                          capture_output=True, 
                          text=True)

print(f"Salida: {resultado.stdout}")
print(f"Código de retorno: {resultado.returncode}")
print(f"Error (si lo hay): {resultado.stderr}")

Salida: Hola desde subprocess

Código de retorno: 0
Error (si lo hay): 


## 2. Listar Archivos con `ls` (Unix/Linux)

In [2]:
# Listar archivos del directorio actual
try:
    resultado = subprocess.run(['ls', '-lh'], 
                              capture_output=True, 
                              text=True,
                              check=True)
    print("Salida de 'ls -lh':")
    print(resultado.stdout)
except FileNotFoundError:
    print("❌ Comando 'ls' no disponible (probablemente Windows)")
    print("En Windows usaríamos 'dir'")
except subprocess.CalledProcessError as e:
    print(f"❌ Error al ejecutar el comando: {e}")

Salida de 'ls -lh':
total 144K
-rw-rw-r-- 1 user user 15K oct 21 10:29 demo_00_live.ipynb
-rw-rw-r-- 1 user user 13K oct 20 19:47 demo_01_modulo_os.ipynb
-rw-rw-r-- 1 user user 33K oct 21 11:30 demo_02_subprocess.ipynb
-rw-rw-r-- 1 user user 16K oct 20 19:55 demo_03_pathlib.ipynb
-rw-rw-r-- 1 user user 19K oct 20 19:55 demo_04_shutil.ipynb



## 3. Comando con Pipe (tubería)

Un **pipe** (`|`) conecta la salida de un comando con la entrada de otro:

```bash
ls -la | grep demo
# └─stdout─→ stdin─┘
```

**Diagrama visual:**
```
┌──────────┐  stdout   ┌──────────┐
│  ls -la  ├──────────►│   grep   │
└──────────┘   (pipe)  │   demo   │
                       └──────────┘
```

En Python, lo implementamos ejecutando ambos comandos y pasando la salida (`stdout`) del primero como entrada (`stdin`) del segundo.

**💡 `text=True`**: Convierte la salida a string automáticamente. Sin esta opción, tendríamos que usar `.decode('utf-8')` para convertir de bytes a texto.

In [3]:
# Equivalente a: ls -la | grep demo
try:
    # Paso 1: Ejecutar ls -la y capturar su salida
    p1 = subprocess.run(['ls', '-la'], 
                       capture_output=True, 
                       text=True)  # text=True → str (no bytes)
    
    # Paso 2: Pasar la salida de p1 como entrada a grep
    p2 = subprocess.run(['grep', 'demo'], 
                       input=p1.stdout,      # stdout de p1 → stdin de p2
                       capture_output=True, 
                       text=True)
    
    print("Archivos que contienen 'demo' en su nombre:")
    print(p2.stdout)
    
except FileNotFoundError:
    print("❌ Comandos no disponibles en este sistema")

Archivos que contienen 'demo' en su nombre:
-rw-rw-r--  1 user user 15351 oct 21 10:29 demo_00_live.ipynb
-rw-rw-r--  1 user user 12886 oct 20 19:47 demo_01_modulo_os.ipynb
-rw-rw-r--  1 user user 33501 oct 21 11:30 demo_02_subprocess.ipynb
-rw-rw-r--  1 user user 16107 oct 20 19:55 demo_03_pathlib.ipynb
-rw-rw-r--  1 user user 18500 oct 20 19:55 demo_04_shutil.ipynb



## 4. Ejecutar Comando Shell con `shell=True`

**`shell=True`** ejecuta el comando a través del shell del sistema (bash, sh, cmd.exe), permitiendo:
- Variables de entorno: `$USER`, `$HOME`
- Pipes: `|`
- Redirecciones: `>`, `>>`
- Operadores lógicos: `&&`, `||`

### ⚠️ Riesgo de Seguridad: Inyección de Comandos

**Ejemplo peligroso:**
```python
# ❌ PELIGROSO - No validar entrada del usuario
nombre = input("Archivo: ")  # Usuario escribe: file.txt; rm -rf /
subprocess.run(f"cat {nombre}", shell=True)  # ¡Ejecuta rm -rf /!
```

**Solución segura:**
```python
# ✅ SEGURO - Lista de argumentos (no shell)
subprocess.run(['cat', nombre])  # El ; se trata como parte del nombre
                                 # No se interpreta como comando
```

**Regla de oro:** 
- ✅ Usa `shell=True` solo cuando necesites características del shell
- ❌ **NUNCA** con datos de usuario sin validar
- ✅ Prefiere listas de argumentos cuando sea posible

In [4]:
# Con shell=True se puede usar la sintaxis completa del shell
resultado = subprocess.run('echo "Usuario: $USER" && echo "Home: $HOME"',
                          shell=True,
                          capture_output=True,
                          text=True)

print("Salida con shell=True:")
print(resultado.stdout)

# Sin shell=True no funciona (no interpreta $USER ni &&)
try:
    resultado2 = subprocess.run(['echo', '"Usuario: $USER" && echo "Home: $HOME"'],
                               capture_output=True,
                               text=True)
    print("\nSalida sin shell=True (literal):")
    print(resultado2.stdout)
except Exception as e:
    print(f"Error: {e}")

Salida con shell=True:
Usuario: user
Home: /home/user


Salida sin shell=True (literal):
"Usuario: $USER" && echo "Home: $HOME"



## 5. Capturar Salida en Tiempo Real

In [5]:
# Ejecutar comando y mostrar salida línea por línea
import sys

try:
    # Usar Popen para control más fino
    proceso = subprocess.Popen(['ls', '-1'],
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               text=True)
    
    print("Archivos (uno por línea):")
    for linea in proceso.stdout:
        print(f"  📄 {linea.strip()}")
    
    proceso.wait()
    print(f"\nCódigo de retorno: {proceso.returncode}")
    
except FileNotFoundError:
    print("❌ Comando no disponible")

Archivos (uno por línea):
  📄 demo_00_live.ipynb
  📄 demo_01_modulo_os.ipynb
  📄 demo_02_subprocess.ipynb
  📄 demo_03_pathlib.ipynb
  📄 demo_04_shutil.ipynb

Código de retorno: 0


## 6. Verificar Código de Retorno

In [6]:
# Comando que falla
resultado = subprocess.run(['ls', 'archivo_que_no_existe'],
                          capture_output=True,
                          text=True)

if resultado.returncode == 0:
    print("✅ Comando ejecutado correctamente")
    print(resultado.stdout)
else:
    print(f"❌ Comando falló con código {resultado.returncode}")
    print(f"Error: {resultado.stderr}")

❌ Comando falló con código 2
Error: ls: no se puede acceder a 'archivo_que_no_existe': No existe el archivo o el directorio



## 7. Ejecutar con Timeout

El parámetro `timeout` limita el tiempo de ejecución. Si el comando excede el límite, se lanza `TimeoutExpired`.

In [7]:
import time

# Caso 1: Comando que completa dentro del timeout
print("✅ Caso 1: Comando rápido (2s) con timeout de 5s")
try:
    resultado = subprocess.run(['sleep', '2'],
                              timeout=5,
                              capture_output=True)
    print("   Comando completado correctamente")
except subprocess.TimeoutExpired:
    print("   ❌ Timeout excedido")
except FileNotFoundError:
    print("   ℹ️ Comando 'sleep' no disponible (Windows)")

✅ Caso 1: Comando rápido (2s) con timeout de 5s
   Comando completado correctamente
   Comando completado correctamente


In [8]:
# Caso 2: Comando que EXCEDE el timeout
print("⏳ Caso 2: Comando lento (10s) con timeout de 3s")
inicio = time.time()

try:
    resultado = subprocess.run(['sleep', '10'],
                              timeout=3,  # Solo 3 segundos
                              capture_output=True)
    print("   ✅ Comando completado")
except subprocess.TimeoutExpired as e:
    transcurrido = time.time() - inicio
    print(f"   ❌ Timeout excedido: {e.timeout}s")
    print(f"   Proceso terminado tras {transcurrido:.1f}s")
except FileNotFoundError:
    print("   ℹ️ Comando 'sleep' no disponible (Windows)")

⏳ Caso 2: Comando lento (10s) con timeout de 3s
   ❌ Timeout excedido: 3s
   Proceso terminado tras 3.0s
   ❌ Timeout excedido: 3s
   Proceso terminado tras 3.0s


## 8. Obtener Información del Sistema

In [9]:
# Obtener información del sistema con comandos Unix/Linux

# 1. Nombre del host
try:
    resultado = subprocess.run(['hostname'],
                              capture_output=True,
                              text=True)
    print(f"📍 Nombre del host: {resultado.stdout.strip()}")
except FileNotFoundError:
    print("❌ Comando 'hostname' no disponible")

# 2. Usuario actual
try:
    resultado = subprocess.run(['whoami'],
                              capture_output=True,
                              text=True)
    print(f"👤 Usuario actual: {resultado.stdout.strip()}")
except FileNotFoundError:
    print("❌ Comando 'whoami' no disponible")

📍 Nombre del host: VMLinuxMint22
👤 Usuario actual: user


## 📚 Resumen de `subprocess`

### Funciones principales:

| Función | Descripción |
|---------|-------------|
| `subprocess.run()` | Ejecutar comando y esperar | 
| `subprocess.Popen()` | Control más fino del proceso |
| `capture_output=True` | Capturar stdout y stderr |
| `text=True` | Devolver strings en vez de bytes |
| `check=True` | Lanzar excepción si el comando falla |
| `timeout=N` | Límite de tiempo en segundos |
| `shell=True` | Ejecutar en el shell (⚠️ cuidado con seguridad) |

### Atributos del resultado:

- `.stdout` - Salida estándar
- `.stderr` - Salida de error
- `.returncode` - Código de retorno (0 = éxito)

### ⚠️ Consideraciones de seguridad:

1. **Evitar `shell=True`** cuando sea posible
2. **No pasar datos no confiables** directamente a comandos
3. **Validar entradas** antes de usarlas en comandos
4. **Usar listas** en lugar de strings para comandos

---

### 📝 Para temas avanzados:

Ver **`demo_05_adicional_ejecucion.ipynb`** para ejemplos de:
- Ejecutar ejecutables específicos del sistema
- Comandos con privilegios elevados (sudo)